In [ ]:
# Import modules
import matplotlib.pyplot as plt 
from matplotlib.colors import to_rgb
import pandas as pd
from scripts.utils.io import load_dict
from glob import glob
import numpy as np

# Load target metrics
metrics = {
    'giant': load_dict(snakemake.input[0]),
    'miniature': load_dict(snakemake.input[1])
}

metrics = pd.DataFrame(metrics).T

# Load trajectories
files = glob(f"{snakemake.input[2]}/*.parquet")

trajectories = {}
for file in files:
    index = int(file.split('/')[-1].split('_')[-1].split('.')[-2])
    trajectories[index] = pd.read_parquet(file)
    trajectories[index].reset_index(inplace=True)

# Calculate additional metrics
metrics['assortativity_norm'] = (metrics['assortativity'] + 1)/2

for metric in ['density','clustering','assortativity_norm']:
    for trajectory in trajectories.values():
        name = 'error_' + metric 
        trajectory[name] = np.abs(trajectory[metric] - metrics.loc['giant'][metric]) / metrics.loc['giant'][metric]

In [ ]:
# Visualize metrics in planes
fig, axes = plt.subplots(1,3,figsize=(9,3),dpi=300)

def plot_trajectories(metric_1, 
                      metric_2,
                      ax):
    # Label plot
    ax.set_xlabel(metric_1.capitalize())
    ax.set_ylabel(metric_2.capitalize())
    
    # Plot trajectories
    cmap = plt.get_cmap('tab10')
    for replica, trajectory in trajectories.items():
        # Calculate color
        n = trajectory[metric_1].shape[0]
        color = np.zeros((n,4))

        # Populate channels
        for i, channel in enumerate(to_rgb(cmap(replica))):
            color[:,i] = channel

        # Populate alpha
        color[:,3] = (np.arange(n) / n)**4
        
        ax.scatter(trajectory[metric_1],
                   trajectory[metric_2],
                   0.5,
                   c=color,
                   linewidth=0)
        
    # Plot target
    ax.scatter(metrics.loc['giant',metric_1],
               metrics.loc['giant',metric_2],
               10,
               marker='x',
               color='r')

    # Plot generated graph
    ax.scatter(metrics.loc['miniature',metric_1],
               metrics.loc['miniature',metric_2],
               10,
               marker='x',
               color='k')

axes[1].set_title("Replica Trajectories")

plot_trajectories('density','clustering',axes[0])
axes[0].set_ylim([0,1])

plot_trajectories('clustering','assortativity_norm',axes[1])
axes[1].set_xlim([0,1])
axes[1].set_ylim([0,1])

plot_trajectories('assortativity_norm','density',axes[2])
axes[2].set_xlim([0,1])

plt.tight_layout()
plt.savefig(snakemake.output[0],bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(5,1,figsize=(6,15),dpi=300)

def plot_trajectories(metric,
                      ax):
    
    for i in range(len(trajectories.keys())):
        ax.plot(trajectories[i][metric],linewidth=0.5)

for i, metric in enumerate(['Beta','Energy','error_density','error_clustering','error_assortativity_norm']):
    plot_trajectories(metric,axes[i])
    axes[i].set_ylabel(metric.capitalize())

    if i != 4:
        axes[i].set_xticks([])
    else:
        axes[i].set_xlabel("Step")

axes[0].set_title("Metric trajectories")
axes[0].legend([f"Replica {num}" for num in range(len(trajectories.keys()))])

plt.tight_layout()
plt.savefig(snakemake.output[1],bbox_inches='tight')